# Streamageddon
Spotify's armageddon investments and data science meet
## Preprocessing

In [3]:
%pip install pandas==2.3.2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

print("Hello, World!")

Hello, World!
